##### updating on 9/27/2018 to work again

In [ ]:
from lendingclub.csv_dl_preparation

##### writing script

In [285]:
%%writefile download_prep.py
# driver download https://github.com/mozilla/geckodriver/releases
# extracted geckodriver to /usr/local/bin in ubuntu
import os
import sys
import subprocess
import time
from stat import S_ISDIR, ST_CTIME, ST_MODE
from shutil import copytree, rmtree
import pause
from selenium.webdriver import Chrome
from selenium.webdriver.chrome import webdriver as chrome_webdriver
from selenium.webdriver.support.ui import Select
import lendingclub.account_info as acc_info


class DriverBuilder():
    # https://stackoverflow.com/questions/45631715/downloading-with-chrome-headless-and-selenium
    def get_driver(self, download_location=None, headless=False):

        driver = self._get_chrome_driver(download_location, headless)

        driver.set_window_size(1400, 700)

        return driver

    def _get_chrome_driver(self, download_location, headless):
        chrome_options = chrome_webdriver.Options()
        if download_location:
            prefs = {'download.default_directory': download_location,
                     'download.prompt_for_download': False,
                     'download.directory_upgrade': True,
                     'safebrowsing.enabled': False,
                     'safebrowsing.disable_download_protection': True}

            chrome_options.add_experimental_option('prefs', prefs)

        if headless:
            chrome_options.add_argument("--headless")

        driver_path = '/usr/local/bin/chromedriver'

        if sys.platform.startswith("win"):
            driver_path += ".exe"

        driver = Chrome(executable_path=driver_path,
                        chrome_options=chrome_options)

        if headless:
            self.enable_download_in_headless_chrome(driver, download_location)

        return driver

    def enable_download_in_headless_chrome(self, driver, download_dir):
        """
        there is currently a "feature" in chrome where
        headless does not allow file download: https://bugs.chromium.org/p/chromium/issues/detail?id=696481
        This method is a hacky work-around until the official chromedriver support for this.
        Requires chrome version 62.0.3196.0 or above.
        """

        # add missing support for chrome "send_command"  to selenium webdriver
        driver.command_executor._commands["send_command"] = (
            "POST", '/session/$sessionId/chromium/send_command')

        params = {'cmd': 'Page.setDownloadBehavior', 'params': {
            'behavior': 'allow', 'downloadPath': download_dir}}
        command_result = driver.execute("send_command", params)
        print("response from browser:")
        for key in command_result:
            print("result:" + key + ":" + str(command_result[key]))


def download_csvs(download_path):
    '''
    downloads all loan_info csvs and pmt_history csv
    '''
    # setup constants
    email = acc_info.email_throwaway
    password = acc_info.password_throwaway
    url_dl = "https://www.lendingclub.com/info/download-data.action"
    url_signin = "https://www.lendingclub.com/auth/login"
    url_pmt_hist = "https://www.lendingclub.com/site/additional-statistics"

    d_builder = DriverBuilder()
    driver = d_builder.get_driver(
        download_location=download_path, headless=True)
    # sign in
    driver.get(url_signin)

    email_box = driver.find_element_by_xpath(
        '/html/body/div[2]/div[1]/div[2]/form[1]/label[1]/input')
    password_box = driver.find_element_by_xpath(
        '/html/body/div[2]/div[1]/div[2]/form[1]/label[2]/input')

    pause.milliseconds(1000)
    email_box.send_keys(email)
    password_box.send_keys(password)

    button = driver.find_element_by_xpath(
        '/html/body/div[2]/div[1]/div[2]/form[1]/button')
    button.click()
    pause.milliseconds(1500)

    # download loan_info
    driver.get(url_dl)
    download_btn = driver.find_element_by_xpath(
        '//*[@id="currentLoanStatsFileName"]')

    select = driver.find_element_by_xpath(
        '//*[@id="loanStatsDropdown"]')  # get the select element
    options = select.find_elements_by_tag_name(
        "option")  # get all the options into a list

    options_dict = {}
    for option in options:  # iterate over the options, place attribute value in list
        options_dict[option.get_attribute("value")] = option.text

    for opt_val, text in options_dict.items():
        print("starting download on option {0}, {1}".format(opt_val, text))

        select = driver.find_element_by_xpath(
            '//*[@id="loanStatsDropdown"]')
        selection = Select(select)
        selection.select_by_value(opt_val)
        download_btn.click()
        pause.milliseconds(2000)

    # payment history downloads
    driver.get(url_pmt_hist)

    pmt_history = driver.find_element_by_link_text(
        'All payments (includes payments made to investors and to LendingClub)')
    pmt_history.click()

    # wait for all downloads to finish
    while True:
        if len(os.listdir(download_path)) != (
                len(options_dict) + 1):  # +1 for one pmt history file
            time.sleep(5)
            print('waiting for all csv downloads to start')
            continue
        else:
            files = os.listdir(download_path)
            k = 0
            time.sleep(5)
            print('checking/waiting for all csv downloads to finish')
            for filename in files:
                if 'crdownload' in filename:
                    time.sleep(60)
                else:
                    k += 1
    #                 print(k)
            if k == len(files):
                time.sleep(2)
                break

    print('done downloading')
    driver.close()
    return True


def get_hashes(path):
    hashes = {}
    files = os.listdir(path)
    for file_ in files:
        a = subprocess.check_output(
            'shasum -a 256 {0}'.format(path + '/' + file_), shell=True)
        hashes[file_] = a.split()[0]
    return hashes


def check_file_changes(csv_folders, just_dled_hashes):
    need_to_clean = False
    try:
        previous_dled_hashes = get_hashes(csv_folders[-2][1])

        # compare new download to previous download
        # check for added or deleted files
        dne_files = set(just_dled_hashes.keys()).intersection(
            set(previous_dled_hashes.keys()))
        add_files = set(previous_dled_hashes.keys()).intersection(
            set(just_dled_hashes.keys()))
        if len(just_dled_hashes) != len(previous_dled_hashes):
            need_to_clean = True
            print("Compared to the previous time new csv's were downloaded, the following files were deleted: \n {0}".format(
                dne_files))
            print("Compared to the previous time new csv's were downloaded, the following files are new additions: \n {0}".format(
                add_files))
        else:
            print('No files were added or deleted since previous downloading of csvs')

        # check for shasum256 changes
        changed_files = []
        for key in just_dled_hashes.keys():
            if previous_dled_hashes[key] != just_dled_hashes[key]:
                changed_files.append()
        if len(changed_files) == 0:
            print('There are no changes to previous downloaded lending club csvs (loan_info and pmt_hist) via shasum256 hashes')
        else:
            need_to_clean = True
            print('Compared to the previous data download, the shasum256 hashes changed for the following files: {0}'.format(
                changed_files))

    except IndexError:
        need_to_clean = True
        print('Could not find previously download directory? This is probably your first time downloading the csvs or the first download to a new path.')

    return need_to_clean


def get_sorted_creationtime_dirs(ppath):
    csv_folders = [os.path.join(ppath, fn) for fn in os.listdir(
        ppath) if fn not in ['archived_csvs', 'working_csvs']]
    csv_folders = [(os.stat(path), path) for path in csv_folders]
    csv_folders = [(stat[ST_CTIME], path)
                   for stat, path in csv_folders if S_ISDIR(stat[ST_MODE])]
    return sorted(csv_folders)


def archiver(archive_flag, ppath, archiver_dir=None):
    archiver_dir = os.path.join(os.path.expanduser(
        '~'), 'all_data', 'lendingclub', 'csvs', 'archived_csvs') if not archiver_dir else archiver_dir
    os.makedirs(archiver_dir, exist_ok=True)
    if archive_flag:
        just_dled = get_sorted_creationtime_dirs(ppath)[-1][1]
        newest_folder = os.path.split(just_dled)[1]
        copytree(just_dled, os.path.join(archiver_dir, newest_folder))
        print('copied most recent download of csvs to archiver_dir')


def cleaner(ppath):
    just_dled = os.path.split(get_sorted_creationtime_dirs(ppath)[-1][1])[1]
    keep_dirs = ['archived_csvs', 'working_csvs', just_dled]
    for tree in os.listdir(ppath):
        if tree not in keep_dirs:
            rmtree(os.path.join(ppath, tree))
            print('removing old dirs {0}'.format(tree))
    return just_dled

Overwriting download_prep.py


In [196]:
# rm -rf ls /home/justin/all_data/lendingclub/csvs/archived_csvs/

In [282]:
ls /home/justin/all_data/lendingclub/csvs

archived_csvs/  csvs_2018_09_30_22h_06m_24s/


In [283]:
ls /home/justin/all_data/lendingclub/csvs/csvs_2018_09_30_13h_41m_58s/

ls: cannot access '/home/justin/all_data/lendingclub/csvs/csvs_2018_09_30_13h_41m_58s/': No such file or directory


In [284]:
ls /home/justin/all_data/lendingclub/csvs/csvs_2018_09_30_22h_06m_24s/

All_Payments_2018_09.zip           LoanStats_securev1_2016Q4.csv.zip
LoanStats3a_securev1.csv.zip       LoanStats_securev1_2017Q1.csv.zip
LoanStats3b_securev1.csv.zip       LoanStats_securev1_2017Q2.csv.zip
LoanStats3c_securev1.csv.zip       LoanStats_securev1_2017Q3.csv.zip
LoanStats3d_securev1.csv.zip       LoanStats_securev1_2017Q4.csv.zip
LoanStats_securev1_2016Q1.csv.zip  LoanStats_securev1_2018Q1.csv.zip
LoanStats_securev1_2016Q2.csv.zip  LoanStats_securev1_2018Q2.csv.zip
LoanStats_securev1_2016Q3.csv.zip


In [280]:
ls /home/justin/all_data/lendingclub/csvs/archived_csvs/csvs_2018_09_30_13h_41m_58s/

All_Payments_2018_09.zip           LoanStats_securev1_2016Q4.csv.zip
LoanStats3a_securev1.csv.zip       LoanStats_securev1_2017Q1.csv.zip
LoanStats3b_securev1.csv.zip       LoanStats_securev1_2017Q2.csv.zip
LoanStats3c_securev1.csv.zip       LoanStats_securev1_2017Q3.csv.zip
LoanStats3d_securev1.csv.zip       LoanStats_securev1_2017Q4.csv.zip
LoanStats_securev1_2016Q1.csv.zip  LoanStats_securev1_2018Q1.csv.zip
LoanStats_securev1_2016Q2.csv.zip  LoanStats_securev1_2018Q2.csv.zip
LoanStats_securev1_2016Q3.csv.zip


In [256]:
# rmtree(os.path.join(os.getcwd(),'test'), ignore_errors=True)

In [273]:
%%writefile download_and_check_csvs.py
import os
import time
import lendingclub.csv_dl_preparation.download_prep as dp

ppath = '/home/justin/all_data/lendingclub/csvs'
now = time.strftime("%Y_%m_%d_%Hh_%Mm_%Ss")
download_path = ppath + '/csvs_' + now
os.mkdir(download_path)

# download csvs
dp.download_csvs(download_path)

# calculate shasum256 hash on just downloaded csvs
just_dled_hashes = dp.get_hashes(download_path)

# get the dirs holding downlaoded csvs by creation time (not archived_csvs dir)
csv_folders = dp.get_sorted_creationtime_dirs(ppath)

# check if compared to previous time, there are changes/additions/deletions in csvs
archive_flag = dp.check_file_changes(csv_folders, just_dled_hashes)

# if something was different (archive_flag), then store a copy of just_downloaded to archives
dp.archiver(archive_flag, ppath)

# removes old downloads
dp.cleaner(ppath)

print('done downloading, checking, and archiving (when necessary) the csv files!!!')

Overwriting download_and_check_csvs.py


##### testing

In [274]:
ls /home/justin/all_data/lendingclub/csvs/

In [281]:
%run download_and_check_csvs.py

response from browser:
result:sessionId:1845b66423db4d476bc9c85d3882c4b8
result:status:0
result:value:None
starting download on option 0, 2007 - 2011
starting download on option 1, 2012 - 2013
starting download on option 2, 2014
starting download on option 3, 2015
starting download on option 4, 2016 Q1
starting download on option 5, 2016 Q2
starting download on option 6, 2016 Q3
starting download on option 7, 2016 Q4
starting download on option 8, 2017 Q1
starting download on option 9, 2017 Q2
starting download on option 10, 2017 Q3
starting download on option 11, 2017 Q4
starting download on option 12, 2018 Q1
starting download on option 13, 2018 Q2
waiting for all csv downloads to start
checking/waiting for all csv downloads to finish
checking/waiting for all csv downloads to finish
checking/waiting for all csv downloads to finish
checking/waiting for all csv downloads to finish
checking/waiting for all csv downloads to finish
checking/waiting for all csv downloads to finish
checking

##### working

In [112]:
# driver download https://github.com/mozilla/geckodriver/releases
# extracted geckodriver to /usr/local/bin in ubuntu
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from tqdm import tqdm
import timeout_decorator
import subprocess
import time
import pause
import re
import json
from stat import S_ISDIR, ST_CTIME, ST_MODE
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
# from selenium.common.exceptions import NoSuchElementException, WebDriverException
from selenium.webdriver import Chrome
from selenium.webdriver.chrome import webdriver as chrome_webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from IPython.display import FileLink
from bs4 import BeautifulSoup
import lendingclub.account_info as acc_info

class DriverBuilder():
    #https://stackoverflow.com/questions/45631715/downloading-with-chrome-headless-and-selenium
    def get_driver(self, download_location=None, headless=False):

        driver = self._get_chrome_driver(download_location, headless)

        driver.set_window_size(1400, 700)

        return driver

    def _get_chrome_driver(self, download_location, headless):
        chrome_options = chrome_webdriver.Options()
        if download_location:
            prefs = {'download.default_directory': download_location,
                     'download.prompt_for_download': False,
                     'download.directory_upgrade': True,
                     'safebrowsing.enabled': False,
                     'safebrowsing.disable_download_protection': True}

            chrome_options.add_experimental_option('prefs', prefs)

        if headless:
            chrome_options.add_argument("--headless")

        driver_path = '/usr/local/bin/chromedriver'

        if sys.platform.startswith("win"):
            driver_path += ".exe"

        driver = Chrome(executable_path=driver_path, chrome_options=chrome_options)

        if headless:
            self.enable_download_in_headless_chrome(driver, download_location)

        return driver

    def enable_download_in_headless_chrome(self, driver, download_dir):
        """
        there is currently a "feature" in chrome where
        headless does not allow file download: https://bugs.chromium.org/p/chromium/issues/detail?id=696481
        This method is a hacky work-around until the official chromedriver support for this.
        Requires chrome version 62.0.3196.0 or above.
        """

        # add missing support for chrome "send_command"  to selenium webdriver
        driver.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')

        params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': download_dir}}
        command_result = driver.execute("send_command", params)
        print("response from browser:")
        for key in command_result:
            print("result:" + key + ":" + str(command_result[key]))
            
def download_csvs():
    '''
    downloads all loan_info csvs and pmt_history csv
    '''
    # setup constants
    email = acc_info.email_throwaway
    password = acc_info.password_throwaway
    url_dl = "https://www.lendingclub.com/info/download-data.action"
    url_signin = "https://www.lendingclub.com/auth/login"
    url_pmt_hist = "https://www.lendingclub.com/site/additional-statistics"
    
    d_builder = DriverBuilder()
    driver = d_builder.get_driver(download_location = download_path, headless=True)
    # sign in
    driver.get(url_signin)

    email_box = driver.find_element_by_xpath('/html/body/div[2]/div[1]/div[2]/form[1]/label[1]/input')
    password_box = driver.find_element_by_xpath('/html/body/div[2]/div[1]/div[2]/form[1]/label[2]/input')

    pause.milliseconds(1000)
    email_box.send_keys(email)
    password_box.send_keys(password)

    button = driver.find_element_by_xpath('/html/body/div[2]/div[1]/div[2]/form[1]/button')
    button.click()
    pause.milliseconds(1500)
    
    # download loan_info
    driver.get(url_dl)
    download_btn = driver.find_element_by_xpath('//*[@id="currentLoanStatsFileName"]')

    select = driver.find_element_by_xpath(
        '//*[@id="loanStatsDropdown"]')  #get the select element
    options = select.find_elements_by_tag_name(
        "option")  #get all the options into a list

    options_dict = {}
    for option in options:  #iterate over the options, place attribute value in list
        options_dict[option.get_attribute("value")] = option.text

    for opt_val, text in options_dict.items():
        print("starting download on option {0}, {1}".format(opt_val, text))

        select = driver.find_element_by_xpath(
        '//*[@id="loanStatsDropdown"]')
        selection = Select(select)
        selection.select_by_value(opt_val)
        download_btn.click()
        pause.milliseconds(2000)

    # payment history downloads            
    driver.get(url_pmt_hist)

    pmt_history = driver.find_element_by_link_text(
        'All payments (includes payments made to investors and to LendingClub)')
    pmt_history.click()
    
    # wait for all downloads to finish
    while True:
        if len(os.listdir(download_path)) != (
                len(options_dict) + 1):  #+1 for one pmt history file
            time.sleep(5)
            print('waiting for all csv downloads to start')
            continue
        else:
            files = os.listdir(download_path)
            k = 0
            time.sleep(5)
            print('checking/waiting for all csv downloads to finish')
            for filename in files:
                if 'crdownload' in filename:
                    time.sleep(30)
                else:
                    k += 1
    #                 print(k)
            if k == len(files):
                time.sleep(2)
                break

    print('done downloading')
    driver.close()
    return True

def get_hashes(path):
    hashes = {}
    files = os.listdir(path)
    for file_ in files:
        a = subprocess.check_output(
            'shasum -a 256 {0}'.format(path + '/' + file_), shell=True)
        hashes[file_] = a.split()[0]
    return hashes

def check_file_changes():
    try:
        previous_dled_hashes = get_hashes(csv_folders[-2][1])

        # compare new download to previous download
        dne_files = set(just_dled_hashes.keys()).intersection(set(previous_dled_hashes.keys()))
        add_files = set(previous_dled_hashes.keys()).intersection(set(just_dled_hashes.keys()))
        if len(just_dled_hashes) != len(previous_dled_hashes):
            print("Compared to the previous time new csv's were downloaded, the following files were deleted: \n {0}".format(dne_files))
            print("Compared to the previous time new csv's were downloaded, the following files are new additions: \n {0}".format(add_files))
        else:
            print('No files were added or deleted since previous downloading of csvs')


        changed_files = []
        for key in just_dled_hashes.keys():
            if previous_dled_hashes[key] != just_dled_hashes[key]:
                changed_files.append()
        if len(changed_files) == 0:
            print('There are no changes to previous downloaded lending club csvs (loan_info and pmt_hist) via shasum256 hashes')
        else:
            print('Compared to the previous data download, the shasum256 hashes changed for the following files: {0}'.format(changed_files))

    except IndexError:
        print('Could not find previously download directory? This is probably your first time downloading the csvs or the first download to a new path.')

In [6]:
# chrome_options = webdriver.ChromeOptions()

ppath = '/home/justin/all_data/lendingclub/csvs'
now = time.strftime("%Y_%m_%d_%Hh_%Mm_%Ss")
download_path = ppath + '/csvs_' + now
os.mkdir(download_path)

# prefs = {'download.default_directory': '{0}'.format(full_path), 'download.prompt_for_download': False,}
# chrome_options.add_experimental_option('prefs', prefs)
# chrome_options.add_argument('--headless')
# driver = webdriver.Chrome('/home/justin/lendingclub/csv_dl_preparation/chromedriver', chrome_options=chrome_options)

In [97]:
just_dled_hashes = get_hashes(download_path)

In [103]:
csv_folders = [os.path.join(ppath, fn) for fn in os.listdir(ppath)]
csv_folders = [(os.stat(path), path) for path in csv_folders]
csv_folders = [(stat[ST_CTIME], path)
           for stat, path in csv_folders if S_ISDIR(stat[ST_MODE])]
csv_folders = sorted(csv_folders)

In [111]:
check_file_changes()

No files were added or deleted since previous downloading of csvs
There are no changes to previous downloaded lending club csvs (loan_info and pmt_hist) via shasum256 hashes


In [107]:
# try:
#     previous_dled_hashes = get_hashes(csv_folders[-2][1])

#     # compare new download to previous download
#     dne_files = set(just_dled_hashes.keys()).intersection(set(previous_dled_hashes.keys()))
#     add_files = set(previous_dled_hashes.keys()).intersection(set(just_dled_hashes.keys()))
#     if len(just_dled_hashes) != len(previous_dled_hashes):
#         print("Compared to the previous time new csv's were downloaded, the following files were deleted: \n {0}".format(dne_files))
#         print("Compared to the previous time new csv's were downloaded, the following files are new additions: \n {0}".format(add_files))
#     else:
#         print('No files were added or deleted since previous downloading of csvs')
        
        
#     changed_files = []
#     for key in just_dled_hashes.keys():
#         if previous_dled_hashes[key] != just_dled_hashes[key]:
#             changed_files.append()
#     if len(changed_files) == 0:
#         print('There are no changes to previous downloaded lending club csvs (loan_info and pmt_hist) via shasum256 hashes')
#     else:
#         print('Compared to the previous data download, the shasum256 hashes changed for the following files: {0}'.format(changed_files))

# except IndexError:
#     print('Could not find previously download directory? This is probably your first time downloading the csvs or the first download to a new path.')

No files were added or deleted since previous downloading of csvs
There are no changes to previous downloaded lending club csvs (loan_info and pmt_hist) via shasum256 hashes


In [31]:
previous_dled_hashes

{'All_Payments_2018_09.zip': b'6bd72d05817a53e890e99dba73dae4f32148842b56de32a08d494e9762bb9e7b',
 'LoanStats3a_securev1.csv.zip': b'260de16b5b0719288a41f1a038345c305adb1ba58c0b8589c9b21c010d7918d6',
 'LoanStats3b_securev1.csv.zip': b'80307b33d5f2802c616afba0e7702d04f92ec0a1cb74ff53df46cbb8f66ae235',
 'LoanStats3c_securev1.csv.zip': b'f567058a32680313e12755238cfa87db526ea6446a9723f88e233c8482fb4a4a',
 'LoanStats3d_securev1.csv.zip': b'0e932b1e6c17e457f99cee30afe3ca09bdb7dc065bafc8650219d183896a7d73',
 'LoanStats_securev1_2016Q1.csv.zip': b'0897da22792bc0dba8836992f788d1f7f0e6f429c2ed89aca1ca87bd127a125b',
 'LoanStats_securev1_2016Q2.csv.zip': b'6a431ccc1feae291e1ed136a2cfeff63e12525dde003cfa15923210b36075054',
 'LoanStats_securev1_2016Q3.csv.zip': b'43ab15c2d7a138e94dae9cf13d9ebda762e9a156629efa9fa2c91948c2aecd29',
 'LoanStats_securev1_2016Q4.csv.zip': b'08570d1ae99d84e6cad8386b479805c203f94510efb395af6c9f1b12df8d05ae',
 'LoanStats_securev1_2017Q1.csv.zip': b'3a17e52e73067e4de78a749123

In [28]:
files = os.listdir(previous_full_path)
files = [file_ for file_ in files if not file_.startswith('.')]

In [30]:
a

b'260de16b5b0719288a41f1a038345c305adb1ba58c0b8589c9b21c010d7918d6  /home/justin/all_data/lendingclub/csvs/csvs_2018_09_28_21h_21m_53s/LoanStats3a_securev1.csv.zip\n'

In [29]:
files

['LoanStats3d_securev1.csv.zip',
 'All_Payments_2018_09.zip',
 'LoanStats3c_securev1.csv.zip',
 'LoanStats_securev1_2016Q2.csv.zip',
 'LoanStats_securev1_2017Q3.csv.zip',
 'LoanStats_securev1_2018Q1.csv.zip',
 'LoanStats_securev1_2017Q4.csv.zip',
 'LoanStats_securev1_2016Q1.csv.zip',
 'LoanStats_securev1_2017Q2.csv.zip',
 'LoanStats_securev1_2016Q3.csv.zip',
 'LoanStats3b_securev1.csv.zip',
 'LoanStats_securev1_2017Q1.csv.zip',
 'LoanStats_securev1_2018Q2.csv.zip',
 'LoanStats_securev1_2016Q4.csv.zip',
 'LoanStats3a_securev1.csv.zip']

In [ ]:
just_dled_hashes = {}
files = os.listdir(download_path)
for file_ in files:
    a = subprocess.check_output(
        'shasum -a 256 {0}'.format(download_path + '/' + file_), shell=True)
    just_dled_hashes[file_] = a.split()[0]

dirs = [d for d in os.listdir(ppath) if os.path.isdir(os.path.join(ppath, d))]
dirs.sort()

In [ ]:
# navigate to the application home page
driver.get()

sign_in = driver.find_element_by_link_text('Sign in')
sign_in.click()

email_box = driver.find_element_by_name('email')
password_box = driver.find_element_by_name('password')

time.sleep(5)
email_box.send_keys(email)
# time.sleep(5)
password_box.send_keys(password)

button = driver.find_element_by_class_name('form-button')
button.click()

In [ ]:
import time
import subprocess
import lendingclub.account_info as acc_info
import dir_constants as dc
from os import mkdir
from selenium import webdriver
# from selenium.webdriver import PhantomJS
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

platform = 'lendingclub'
chrome_options = webdriver.ChromeOptions()
home = dc.home_path
ppath = home + '/justin_tinkering/data_science/lendingclub/lendingclub_csvs'.format(platform)
now = time.strftime("%Y_%m_%d_%Hh_%Mm_%Ss")
full_path = ppath + '/' + 'lc_' + now
os.mkdir(full_path)
email = acc_info.email_throwaway
password = acc_info.password_throwaway

prefs = {'download.default_directory': '{0}'.format(full_path)}
chrome_options.add_experimental_option('prefs', prefs)
# driver_loc = "/home/justin/justin_tinkering/chromedriver"

# create a new Firefox session
driver = webdriver.PhantomJS() #executable_path=driver_loc
driver.implicitly_wait(30)
driver.maximize_window()

# navigate to the application home page
driver.get("https://www.lendingclub.com/info/download-data.action")

sign_in = driver.find_element_by_link_text('Sign in')
sign_in.click()

email_box = driver.find_element_by_name('email')
password_box = driver.find_element_by_name('password')

time.sleep(5)
email_box.send_keys(email)
# time.sleep(5)
password_box.send_keys(password)

button = driver.find_element_by_class_name('form-button')
button.click()

time.sleep(10)

statistics = driver.find_element_by_link_text('Statistics')
statistics.click()

download_data = driver.find_element_by_link_text('Download Data')
download_data.click()

select = driver.find_element_by_id(
    'loanStatsDropdown')  #get the select element
options = select.find_elements_by_tag_name(
    "option")  #get all the options into a list

optionsList = []

for option in options:  #iterate over the options, place attribute value in list
    optionsList.append(option.get_attribute("value"))

for optionValue in optionsList:
    print("starting loop on option %s" % optionValue)

    select = Select(driver.find_element_by_id('loanStatsDropdown'))
    select.select_by_value(optionValue)
    download_btn = driver.find_element_by_id('currentLoanStatsFileName')
    download_btn.click()
    time.sleep(2)

driver.get('https://www.lendingclub.com/site/additional-statistics')
pmt_history = driver.find_element_by_link_text(
    'All payments (includes payments made to investors and to LendingClub)')
pmt_history.click()
time.sleep(2)

while True:
    if len(os.listdir(full_path)) != (
            len(optionsList) + 1):  #+1 for one pmt history file
        time.sleep(5)
    else:
        files = os.listdir(full_path)
        k = 0
        time.sleep(5)
        for filename in files:
            if 'crdownload' in filename:
                print('waiting on downloads to finish.')
                time.sleep(60)
            else:
                k += 1
#                 print(k)
        if k == len(files):
            time.sleep(2)
            break

print('done downloading')

driver.close()

just_dled_hashes = {}
files = os.listdir(full_path)
for file_ in files:
    a = subprocess.check_output(
        'shasum -a 256 {0}'.format(full_path + '/' + file_), shell=True)
    just_dled_hashes[file_] = a.split()[0]

dirs = [d for d in os.listdir(ppath) if os.path.isdir(os.path.join(ppath, d))]
dirs.sort()

try:
    previous_dled = dirs[-2]

    previous_dled_hashes = {}
    previous_full_path = ppath + '/' + previous_dled + '/'
    files = os.listdir(previous_full_path)
    files = [file_ for file_ in files if not file_.startswith('.')]
    for file_ in files:
        a = subprocess.check_output(
            'shasum -a 256 {0}'.format(previous_full_path + file_), shell=True)
        previous_dled_hashes[file_] = a.split()[0]

    k = 0
    if len(just_dled_hashes) != len(previous_dled_hashes):
        k += 1
    else:
        for key in just_dled_hashes.keys():
            if previous_dled_hashes[key] != just_dled_hashes[key]:
                k += 1

    if k == 0:
        print('no change to previous downloaded lending club loan info csvs')
    else:
        print(
            'At least one of the files changed. Probably need to unzip csvs and re-run cleaning scripts.'
        )

except IndexError:
    print('this is probably your first time downloading the csvs.')

In [ ]:
Justinjust_dled_hashes

In [ ]:
previous_dled_hashes

# Below was for storing in s3 bucket

In [ ]:
aws_access_key_id = 
aws_secret_access_key = 

In [ ]:
import boto3

s3 = boto3.resource('s3')

In [ ]:
dirs = [d for d in os.listdir(ppath) if os.path.isdir(os.path.join(ppath,d))]
dirs.sort()
most_recent_dls = dirs[-1]

In [ ]:
files_to_store = os.listdir(ppath+'/'+most_recent_dls)

In [ ]:
ppath+'/'+most_recent_dls

In [ ]:
bucket_name = ''
for file_ in tqdm_notebook(files_to_store):
    data = open('{0}'.format(ppath+'/'+most_recent_dls+'/'+file_), 'rb')
    s3.Bucket(bucket_name).put_object(Key='{0}/{1}'.format(most_recent_dls, file_), Body=data)